In [ ]:
from kafka import KafkaConsumer, KafkaProducer
from detector.detector import get_transform, get_fasterrcnn_model


bootstrap_servers = ['kafka1:9091']
topicName = 'my-test-video'

producer = KafkaProducer(bootstrap_servers = bootstrap_servers)

path = './data/sequence-1/img1'
image_filenames = sorted(os.listdir(path))[:4]



for image_name in :
    producer.send(topicName, )
    producer.flush()


consumer = KafkaConsumer('my-test-video', bootstrap_servers=['kafka1:9091'])

for message in consumer:
    print (message)
                          

def make_detections_on_image_stream(path, model):

    image_id = -1

    vidObj = cv2.VideoCapture(path)

    transform = get_transform(False)
    model = model.eval()

    all_frame_results_array = None

    # checks whether frames were extracted
    success = 1

    while success:

        success, image = vidObj.read()

        image = Image.fromarray(image)

        # img = Image.open('./data/sequence-1/img1/000608.jpg').convert("RGB")

        image_tensor = transform(image, {})
        image_tensor = image_tensor[0].unsqueeze(0).float()

        with torch.no_grad():
            predictions = model(image_tensor)

        for frame in zip(predictions):

            frame = frame[0]
            image_id += 1
            boxes = frame['boxes'].cpu().numpy()
            labels = frame['labels'].cpu().numpy().reshape(-1,1)
            scores = frame['scores'].cpu().numpy().reshape(-1,1)
            image_id_col = np.array([image_id]*scores.shape[0]).reshape(-1,1)
            x = np.array([-1]*scores.shape[0]).reshape(-1,1)
            y = np.array([-1]*scores.shape[0]).reshape(-1,1)
            z = np.array([-1]*scores.shape[0]).reshape(-1,1)

            # Convert bbox coords to tlwh from tlbr
            boxes[:,2] = boxes[:,2]-boxes[:,0]
            boxes[:,3] = boxes[:,3]-boxes[:,1]

            frame_results = np.hstack([image_id_col, labels, boxes, scores, x, y, z])

            if all_frame_results_array is not None:
                all_frame_results_array = np.vstack([all_frame_results_array, frame_results])
            else:
                all_frame_results_array = frame_results

        np.savetxt(os.path.join('det.txt'), all_frame_results_array, delimiter=",")